In [ ]:
!pip install pandas matplotlib prophet

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving retail_sales_dataset.csv to retail_sales_dataset (1).csv


In [ ]:
import pandas as pd
df = pd.read_csv("retail_sales_dataset.csv")
df.head()

,Unnamed: 0,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,0,1,1,2010-02-05,24924.50,False,5.727778,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,0,151315
1,1,1,1,2010-02-12,46039.49,True,3.616667,2.548,0.0,0.0,0.0,0.0,0.0,211.242170,8.106,0,151315
2,2,1,1,2010-02-19,41595.55,False,4.405556,2.514,0.0,0.0,0.0,0.0,0.0,211.289143,8.106,0,151315
3,3,1,1,2010-02-26,19403.54,False,8.127778,2.561,0.0,0.0,0.0,0.0,0.0,211.319643,8.106,0,151315
4,4,1,1,2010-03-05,21827.90,False,8.055556,2.625,0.0,0.0,0.0,0.0,0.0,211.350143,8.106,0,151315


In [ ]:
# Check raw date values first (optional debugging)
print(df['Date'].unique()[:10])

['2010-02-05' '2010-02-12' '2010-02-19' '2010-02-26' '2010-03-05'
 '2010-03-12' '2010-03-19' '2010-03-26' '2010-04-02' '2010-04-09']


In [ ]:
# Convert Date safely with coercion
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce', infer_datetime_format=True)

# Drop rows where conversion failed (NaT)
df = df.dropna(subset=['Date'])

# Filter one time series (Store 1, Dept 1)
store_df = df[(df['Store'] == 1) & (df['Dept'] == 1)].copy()

# Prepare for Prophet
store_df = store_df.rename(columns={'Date': 'ds', 'Weekly_Sales': 'y'})
store_df = store_df[['ds', 'y']].dropna().sort_values('ds')

store_df.head()


<ipython-input-9-25712354d46f>:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce', infer_datetime_format=True)


,ds,y
34,2010-01-10,20094.19
8,2010-02-04,57258.43
21,2010-02-07,16333.14
30,2010-03-09,16241.78
43,2010-03-12,22517.56


In [ ]:
from prophet import Prophet

model = Prophet()
model.fit(store_df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmptxbod00l/yjwv5l8b.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmptxbod00l/ch04x24q.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=94267', 'data', 'file=/tmp/tmptxbod00l/yjwv5l8b.json', 'init=/tmp/tmptxbod00l/ch04x24q.json', 'output', 'file=/tmp/tmptxbod00l/prophet_modelhewc6yb5/prophet_model-20250527083902.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
08:39:02 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
08:39:02 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [ ]:
future = model.make_future_dataframe(periods=12, freq='W')  # Next 12 weeks
forecast = model.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()


,ds,yhat,yhat_lower,yhat_upper
64,2013-02-03,25881.388429,15640.504639,36894.535140
65,2013-02-10,13540.860037,3615.183088,24100.873615
66,2013-02-17,7166.053583,-3028.176212,18279.237550
67,2013-02-24,9045.005841,-1645.263503,19577.795766
68,2013-03-03,14053.590212,4182.465962,24865.041028


In [ ]:
from prophet.plot import plot_plotly, plot_components_plotly

plot_plotly(model, forecast)  # Main forecast plot

In [ ]:
import joblib

joblib.dump(model, 'prophet_sales_model.pkl')

['prophet_sales_model.pkl']